In [13]:
# import library yang dibutuhkan
import tensorflow as tf
from sklearn.model_selection import train_test_split
import shutil
import zipfile,os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image as keras_image
import matplotlib.pyplot as plt
import cv2
from rembg.bg import remove
import easygui
from PIL import Image
from concurrent.futures import ProcessPoolExecutor
import wget

print(tf.__version__)

2.18.0


In [14]:
# check directory train ada isinya atau ga
if os.path.exists('./train'):
    print('Direktori sudah ada isinya')

Direktori sudah ada isinya


# Data Preprocessing

In [19]:
import os
from sklearn.model_selection import train_test_split
from rembg import remove
from PIL import Image


# fungsi untuk menghapus background dari sebuah gambar
def remove_background(input_path, output_path):
    try:
        input_image = Image.open(input_path) # ngebuka filenya
        output_image = remove(input_image) # ngehapus background -> library rembg
        output_image.save(output_path) # nyimpen yang udah diapus ke background
    except Exception as e: # kalo gagal nyimpen yg removed background disimpen di log
        with open("error_log.txt", "a") as log_file:
            log_file.write(f"Error processing {input_path}: {e}\n")


# # fungsi untuk memproses penghapusan background secara rekursif
# def remove_background_serial(input_dir, output_dir):
#     for root, dirs, files in os.walk(input_dir):  # Rekursif ke subfolder
        
#         # root : directory saat ini yang sedang diproses
#         # dirs : subdirectory dalam root
#         # files : file dalam root
        
#         # bikin biar strukturnya masih sama
#         relative_path = os.path.relpath(root, input_dir)
#         current_output_dir = os.path.join(output_dir, relative_path)

#         # root harus ada, kalau gaada bikin dulu
#         if not os.path.exists(current_output_dir):
#             os.makedirs(current_output_dir)

#         # loop setiap directory
#         for filename in files:
#             input_path = os.path.join(root, filename)
#             output_path = os.path.join(current_output_dir, filename)

#             remove_background(input_path, output_path)
#             print(f"Processed: {input_path} -> {output_path}")

def remove_background_serial(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        relative_path = os.path.relpath(root, input_dir)
        current_output_dir = os.path.join(output_dir, relative_path)

        if not os.path.exists(current_output_dir):
            os.makedirs(current_output_dir)

        for filename in files:
            input_path = os.path.join(root, filename)
            output_path = os.path.join(current_output_dir, filename)

            try:
                # Cek apakah input file adalah gambar
                if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    print(f"Skipped non-image file: {filename}")
                    continue

                # Proses penghapusan background
                remove_background(input_path, output_path)
                print(f"Processed: {input_path} -> {output_path}")

            except Exception as e:
                # Log error jika gagal
                print(f"Error processing {input_path}: {e}")


In [20]:
# path dataset asli dan hasil
original_dataset_dir = './train'  # dataset asli
processed_dataset_dir = './processed'  # dataset setelah background dihapus
processed_split_dir = './processed_split'  # direktori hasil pembagian train/validation

# kategori dan daftar subject
categories = ['fall', 'non_fall']
subjects = [f for f in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, f))]

# # ngapus background
# for subject in subjects:
#     for category in categories:
#         input_dir = os.path.join(original_dataset_dir, subject, category)
#         output_dir = os.path.join(processed_dataset_dir, subject, category)

#         if os.path.exists(input_dir):
#             remove_background_serial(input_dir, output_dir)

for subject in subjects[:1]:  # Hanya subject pertama
    for category in categories:
        input_dir = os.path.join(original_dataset_dir, subject, category)
        output_dir = os.path.join(processed_dataset_dir, subject, category)

        if os.path.exists(input_dir):
            remove_background_serial(input_dir, output_dir)


Processed: ./train/subject-2/fall/2_forward_falls/frame024.jpg -> ./processed/subject-2/fall/2_forward_falls/frame024.jpg
Processed: ./train/subject-2/fall/2_forward_falls/frame030.jpg -> ./processed/subject-2/fall/2_forward_falls/frame030.jpg
Processed: ./train/subject-2/fall/2_forward_falls/frame018.jpg -> ./processed/subject-2/fall/2_forward_falls/frame018.jpg
Processed: ./train/subject-2/fall/2_forward_falls/frame019.jpg -> ./processed/subject-2/fall/2_forward_falls/frame019.jpg
Processed: ./train/subject-2/fall/2_forward_falls/frame031.jpg -> ./processed/subject-2/fall/2_forward_falls/frame031.jpg
Processed: ./train/subject-2/fall/2_forward_falls/frame025.jpg -> ./processed/subject-2/fall/2_forward_falls/frame025.jpg


KeyboardInterrupt: 

In [24]:
# # membuat direktori untuk data training dan data validasi
# base_dir = './processed'
# train_dir = os.path.join(base_dir, 'train')
# validation_dir = os.path.join(base_dir, 'val')

# # membuat direktori untuk data training 
# train_fall_dir = os.path.join(train_dir, 'fall') 
# train_non_fall_dir = os.path.join(train_dir, 'non_fall')

# # membuat direktori untuk data validasi
# validation_fall_dir = os.path.join(validation_dir, 'fall')
# validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# # membuat direktori untuk data training dan data validasi
# if not os.path.exists(train_dir):
#     os.mkdir(train_dir)
# if not os.path.exists(validation_dir):
#     os.mkdir(validation_dir)

# if not os.path.exists(train_fall_dir):
#     os.mkdir(train_fall_dir)
# if not os.path.exists(train_non_fall_dir):
#     os.mkdir(train_non_fall_dir)

# if not os.path.exists(validation_fall_dir):
#     os.mkdir(validation_fall_dir)
# if not os.path.exists(validation_non_fall_dir):
#     os.mkdir(validation_non_fall_dir)

In [35]:
# Step 2: Membagi dataset menjadi train dan validation
train_dir = os.path.join(processed_split_dir, 'train')
validation_dir = os.path.join(processed_split_dir, 'val')

train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# Membuat semua direktori jika belum ada
for directory in [train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Step 3: Membagi dataset
train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    # Fall
    fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
    fall_images = [os.path.join(fall_dir, img) for img in os.listdir(fall_dir)] if os.path.exists(fall_dir) else []

    if fall_images:
        train, val = train_test_split(fall_images, test_size=0.4, random_state=42)
        train_fall += train
        validation_fall += val

    # Non-fall
    non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
    non_fall_images = [os.path.join(non_fall_dir, img) for img in os.listdir(non_fall_dir)] if os.path.exists(non_fall_dir) else []

    if non_fall_images:
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=42)
        train_non_fall += train
        validation_non_fall += val

# Step 4: Memindahkan file ke folder train dan validation
def move_files(file_list, target_dir):
    for file_path in file_list:
        filename = os.path.basename(file_path)
        target_path = os.path.join(target_dir, filename)

        if not os.path.exists(target_path):
            os.rename(file_path, target_path)

# Memindahkan data fall
move_files(train_fall, train_fall_dir)
move_files(validation_fall, validation_fall_dir)

# Memindahkan data non-fall
move_files(train_non_fall, train_non_fall_dir)
move_files(validation_non_fall, validation_non_fall_dir)

print("Proses selesai!")

check kosong
check kosong
check kosong
check kosong
check kosong
check kosong
check kosong
check kosong


In [27]:
# # membagi data train dan data validation dengan perbandingan 60:40 

# subjects = [f for f in os.listdir('./processed') if os.path.isdir(os.path.join('./processed', f))]
# train_fall = []
# validation_fall = []

# for subject in subjects:
#     fall_images = os.listdir(os.path.join('./processed', subject, 'fall'))
#     train, val = train_test_split(fall_images, test_size=0.4, random_state=None)
#     train_fall += [os.path.join(subject, 'fall', img) for img in train]
#     validation_fall += [os.path.join(subject, 'fall', img) for img in val]

# # train test split jika direktori belum ada isinya
# if len(os.listdir('./train/fall')) == 0 and len(os.listdir('./train/non_fall')) == 0 :
#     # fall
#     train_fall = os.listdir('./processed/fall')
#     validation_fall = os.listdir('./processed/fall')
#     train_fall, validation_fall = train_test_split(train_fall, test_size = 0.4, random_state=None)

#     # non_fall
#     train_non_fall = os.listdir('./processed/non_fall')
#     validation_non_fall = os.listdir('./processed/non_fall')
#     train_non_fall, validation_non_fall = train_test_split(train_non_fall, test_size = 0.4, random_state=None)

#     # Memindahkan file gambar ke direktori data training (train)
#     for image in train_fall:
#         shutil.move(os.path.join('./processed/fall', image), os.path.join(train_fall_dir, image))
#     for image in train_non_fall:
#         shutil.move(os.path.join('./processed/non_fall', image), os.path.join(train_non_fall_dir, image))

#     # Memindahkan file gambar ke direktori data validasi (validation)
#     for image in validation_fall:
#         shutil.move(os.path.join('./processed/fall', image), os.path.join(validation_fall_dir, image))
#     for image in validation_non_fall:
#         shutil.move(os.path.join('./processed/non_fall', image), os.path.join(validation_non_fall_dir, image))
# else:
#     print('Direktori sudah ada isinya')

In [28]:
# # memindahkan file ke direktori training dan validasi
# for subject, category, image in train_fall:
#     src = os.path.join(processed_dataset_dir, subject, category, image)
#     dst = os.path.join(train_fall_dir, image)
#     shutil.move(src, dst)

# for subject, category, image in validation_fall:
#     src = os.path.join(processed_dataset_dir, subject, category, image)
#     dst = os.path.join(validation_fall_dir, image)
#     shutil.move(src, dst)

# for subject, category, image in train_non_fall:
#     src = os.path.join(processed_dataset_dir, subject, category, image)
#     dst = os.path.join(train_non_fall_dir, image)
#     shutil.move(src, dst)

# for subject, category, image in validation_non_fall:
#     src = os.path.join(processed_dataset_dir, subject, category, image)
#     dst = os.path.join(validation_non_fall_dir, image)
#     shutil.move(src, dst)

# print("Dataset berhasil diproses dan dipisahkan ke dalam direktori training dan validasi.")

Dataset berhasil diproses dan dipisahkan ke dalam direktori training dan validasi.


In [29]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path dataset hasil pemrosesan dan direktori untuk train/validation
processed_dir = './processed'
train_dir = './processed_split/train'
validation_dir = './processed_split/val'

# Subdirektori kategori untuk training dan validation
train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# Membuat semua direktori jika belum ada
for directory in [train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Mengambil daftar subject
subjects = [f for f in os.listdir(processed_dir) if os.path.isdir(os.path.join(processed_dir, f))]

# Membagi data train dan validation
train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    # Fall
    fall_dir = os.path.join(processed_dir, subject, 'fall')
    if os.path.exists(fall_dir):
        fall_images = os.listdir(fall_dir)
        train, val = train_test_split(fall_images, test_size=0.4, random_state=42)
        train_fall += [(os.path.join(fall_dir, img), os.path.join(train_fall_dir, f"{subject}_{img}")) for img in train]
        validation_fall += [(os.path.join(fall_dir, img), os.path.join(validation_fall_dir, f"{subject}_{img}")) for img in val]

    # Non-fall
    non_fall_dir = os.path.join(processed_dir, subject, 'non_fall')
    if os.path.exists(non_fall_dir):
        non_fall_images = os.listdir(non_fall_dir)
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=42)
        train_non_fall += [(os.path.join(non_fall_dir, img), os.path.join(train_non_fall_dir, f"{subject}_{img}")) for img in train]
        validation_non_fall += [(os.path.join(non_fall_dir, img), os.path.join(validation_non_fall_dir, f"{subject}_{img}")) for img in val]

# Memindahkan file ke direktori train dan validation
def move_files(file_list):
    for src, dest in file_list:
        if not os.path.exists(dest):  # Pastikan tidak overwrite
            shutil.move(src, dest)

# Memindahkan data fall
move_files(train_fall)
move_files(validation_fall)

# Memindahkan data non-fall
move_files(train_non_fall)
move_files(validation_non_fall)

print("Proses pembagian train dan validation selesai!")

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path direktori
processed_dataset_dir = './processed'
processed_split_dir = './processed_split'
train_dir = os.path.join(processed_split_dir, 'train')
validation_dir = os.path.join(processed_split_dir, 'val')

# Subdirektori kategori
train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

# Membuat semua direktori jika belum ada
for directory in [train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Mengambil daftar subject
subjects = [f for f in os.listdir(processed_dataset_dir) if os.path.isdir(os.path.join(processed_dataset_dir, f))]

# Membagi dataset menjadi train dan validation
train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    # Fall
    fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
    if os.path.exists(fall_dir):
        fall_images = os.listdir(fall_dir)
        train, val = train_test_split(fall_images, test_size=0.4, random_state=42)
        train_fall += [(subject, 'fall', img) for img in train]
        validation_fall += [(subject, 'fall', img) for img in val]

    # Non-fall
    non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
    if os.path.exists(non_fall_dir):
        non_fall_images = os.listdir(non_fall_dir)
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=42)
        train_non_fall += [(subject, 'non_fall', img) for img in train]
        validation_non_fall += [(subject, 'non_fall', img) for img in val]

# Memindahkan file ke direktori training dan validasi
def move_files(file_list, target_dir):
    for subject, category, image in file_list:
        src = os.path.join(processed_dataset_dir, subject, category, image)
        dst = os.path.join(target_dir, f"{subject}_{image}")  # Tambahkan prefix subject untuk menghindari konflik nama
        if not os.path.exists(dst):  # Hindari overwrite
            shutil.move(src, dst)

# Memindahkan file Fall
move_files(train_fall, train_fall_dir)
move_files(validation_fall, validation_fall_dir)

# Memindahkan file Non-Fall
move_files(train_non_fall, train_non_fall_dir)
move_files(validation_non_fall, validation_non_fall_dir)

print("Dataset berhasil diproses dan dipisahkan ke dalam direktori training dan validasi.")

In [31]:
# menampilkan nama kelas yang ada pada train
os.listdir('./processed/train')

['non_fall', 'fall']

In [32]:
# menampilkan nama kelas yang ada pada train
os.listdir('./processed/val')

['non_fall', 'fall']

# Data Modelling

In [ ]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from sklearn.model_selection import train_test_split
from rembg import remove
from PIL import Image

# 1. Fungsi Preprocessing Gambar
def preprocess_img(img):
    img = cv2.GaussianBlur(img, (5, 5), 0)  # Menghaluskan gambar
    img = tf.image.adjust_contrast(img, 2)  # Menyesuaikan kontras gambar
    return img

# 2. Fungsi Penghapusan Background
def remove_background(input_path, output_path):
    try:
        input_image = Image.open(input_path)
        output_image = remove(input_image)
        output_image.save(output_path)
    except Exception as e:
        with open("error_log.txt", "a") as log_file:
            log_file.write(f"Error processing {input_path}: {e}\n")

def remove_background_serial(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        remove_background(input_path, output_path)

# 3. Path Dataset
original_dataset_dir = './train'
processed_dataset_dir = './processed'
categories = ['fall', 'non_fall']

# Memproses dataset untuk setiap subject dan kategori
subjects = [f for f in os.listdir(original_dataset_dir) if os.path.isdir(os.path.join(original_dataset_dir, f))]
for subject in subjects:
    for category in categories:
        input_dir = os.path.join(original_dataset_dir, subject, category)
        output_dir = os.path.join(processed_dataset_dir, subject, category)
        if os.path.exists(input_dir):
            remove_background_serial(input_dir, output_dir)

# 4. Membagi Data Train dan Validation
base_dir = './processed_split'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
train_fall_dir = os.path.join(train_dir, 'fall')
train_non_fall_dir = os.path.join(train_dir, 'non_fall')
validation_fall_dir = os.path.join(validation_dir, 'fall')
validation_non_fall_dir = os.path.join(validation_dir, 'non_fall')

for directory in [train_dir, validation_dir, train_fall_dir, train_non_fall_dir, validation_fall_dir, validation_non_fall_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

train_fall = []
validation_fall = []
train_non_fall = []
validation_non_fall = []

for subject in subjects:
    fall_dir = os.path.join(processed_dataset_dir, subject, 'fall')
    fall_images = os.listdir(fall_dir) if os.path.exists(fall_dir) else []
    if len(fall_images) > 0:
        train, val = train_test_split(fall_images, test_size=0.4, random_state=42)
        train_fall += [(subject, 'fall', img) for img in train]
        validation_fall += [(subject, 'fall', img) for img in val]

    non_fall_dir = os.path.join(processed_dataset_dir, subject, 'non_fall')
    non_fall_images = os.listdir(non_fall_dir) if os.path.exists(non_fall_dir) else []
    if len(non_fall_images) > 0:
        train, val = train_test_split(non_fall_images, test_size=0.4, random_state=42)
        train_non_fall += [(subject, 'non_fall', img) for img in train]
        validation_non_fall += [(subject, 'non_fall', img) for img in val]

for subject, category, image in train_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_fall_dir, f"{subject}_{image}")
    shutil.move(src, dst)

for subject, category, image in validation_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_fall_dir, f"{subject}_{image}")
    shutil.move(src, dst)

for subject, category, image in train_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(train_non_fall_dir, f"{subject}_{image}")
    shutil.move(src, dst)

for subject, category, image in validation_non_fall:
    src = os.path.join(processed_dataset_dir, subject, category, image)
    dst = os.path.join(validation_non_fall_dir, f"{subject}_{image}")
    shutil.move(src, dst)

# 5. ImageDataGenerator untuk Training dan Validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest',
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    preprocessing_function=preprocess_img,
    brightness_range=[0.5, 1.5],
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# 6. Load dan Augmentasi Gambar untuk Visualisasi
img_path = os.path.join(train_fall_dir, os.listdir(train_fall_dir)[0])  # Ambil gambar pertama dari train/fall
img = tf.keras.preprocessing.image.load_img(img_path)
x = tf.keras.preprocessing.image.img_to_array(img)
x = x.reshape((1,) + x.shape)

plt.figure(figsize=(10, 10))
plt.suptitle('Augmented Images')

i = 0
for batch in train_datagen.flow(x, batch_size=1):
    plt.subplot(3, 3, i + 1)
    plt.imshow(array_to_img(batch[0]))
    i += 1
    if i >= 6:
        break

plt.show()

print("Proses selesai. Dataset telah diproses, dibagi menjadi train dan validation, dan augmentasi telah selesai.")

In [ ]:
# Menggunakan objek ImageDataGenerator sebelumnya untuk mempersiapkan data latih dan validasi

# Data training
train_generator = train_datagen.flow_from_directory(
        train_dir,              # direktori data latih
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=32,          # jumlah sampel yang diproses dalam satu iterasi
        class_mode='binary'     # karena ini adalah klasifikasi 2 kelas (fall dan non-fall), gunakan 'binary'
)

# Data validation
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,         # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=32,          # jumlah sampel yang diproses dalam satu iterasi
        class_mode='binary'     # karena ini adalah klasifikasi 2 kelas (fall dan non-fall), gunakan 'binary'
)


In [ ]:
from tensorflow.keras import layers, models

# Arsitektur model sederhana
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Aktivasi sigmoid untuk klasifikasi biner
])

# Kompilasi model
model.compile(
    loss='binary_crossentropy', # Loss function untuk klasifikasi biner
    optimizer='adam',           # Optimizer
    metrics=['accuracy']        # Metode evaluasi
)

# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,                   # Jumlah epoch
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)
